## Import libraries

In [436]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

## Load and preprocess data

In [437]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# Fill nan values with mode
null_columns1 = train.columns[train.isnull().any()]
for column in null_columns1:
    train[column].fillna(train[column].mode()[0], inplace=True)
    
null_columns2 = test.columns[test.isnull().any()]
for column in null_columns2:
    test[column].fillna(test[column].mode()[0], inplace=True)

In [438]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,120.0,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [439]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,1.0,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


### Encode train data

In [440]:
# encode the categorical values
encoded_gender = pd.get_dummies(train['Gender'])
train = train.drop('Gender', axis = 1)
train = train.join(encoded_gender)

encoded_education = pd.get_dummies(train['Education'])
train = train.drop('Education', axis = 1)
train = train.join(encoded_education)

encoded_area = pd.get_dummies(train['Property_Area'])
train = train.drop('Property_Area', axis = 1)
train = train.join(encoded_area)

le = preprocessing.LabelEncoder()
train['Self_Employed'] = le.fit_transform(train['Self_Employed'])
train['Married'] = le.fit_transform(train['Married'])
train['Loan_Status'] = le.fit_transform(train['Loan_Status'])

train['Dependents'] = train['Dependents'].replace('3+', 3)
train = train.drop('Loan_ID', axis=1)
print(train.shape)

(614, 16)


### Encode test data

In [441]:
# encode the categorical values
encoded_gender = pd.get_dummies(test['Gender'])
test = test.drop('Gender', axis = 1)
test = test.join(encoded_gender)

encoded_education = pd.get_dummies(test['Education'])
test = test.drop('Education', axis = 1)
test = test.join(encoded_education)

encoded_area = pd.get_dummies(test['Property_Area'])
test = test.drop('Property_Area', axis = 1)
test = test.join(encoded_area)

le = preprocessing.LabelEncoder()
test['Self_Employed'] = le.fit_transform(test['Self_Employed'])
test['Married'] = le.fit_transform(test['Married'])

test['Dependents'] = test['Dependents'].replace('3+', 3)
test = test.drop('Loan_ID', axis=1)

print(test.shape)

(367, 15)


In [442]:
# convert to array
label = train['Loan_Status'].values
train = train.drop('Loan_Status', axis=1)
train_data = train.values
test_data = test.values

In [443]:
print(train.shape)

(614, 15)


### Prepare train test data

In [446]:
train_x = train_data
train_y = label
x_test = test_data
print(train_x.shape)
print(train_y.shape)
print(x_test.shape)

(614, 15)
(614,)
(367, 15)


### Prepare validation dataset

In [447]:
split_size = int(x_train.shape[0] * 0.7)
train_x, val_x = train_x[:split_size], train_x[split_size:]
train_y, val_y = train_y[:split_size], train_y[split_size:]

## Model Building

In [448]:
# Setting the size of nodes
input_layer_size = 15
hidden_layer_size = 50
output_layer_size = 1 # Output will be either 1 0r 0

In [473]:
epochs = 1000

In [474]:
# Construct the model
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
    Dense(input_dim = input_layer_size, output_dim = hidden_layer_size, activation = 'relu'),
    Dense(input_dim = hidden_layer_size, output_dim = output_layer_size, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_59 (Dense)             (None, 50)                800       
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 51        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________


/home/rebekkal/miniconda3/envs/rv/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, activation="relu", units=50)`
  
/home/rebekkal/miniconda3/envs/rv/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, activation="sigmoid", units=1)`
  import sys


## Train and Validate

In [475]:
# Train the model
model.fit(train_x, train_y, nb_epoch = epochs, validation_data = (val_x, val_y))

/home/rebekkal/miniconda3/envs/rv/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 429 samples, validate on 185 samples
Epoch 1/1000
429/429 [==============================] - 0s 350us/step - loss: 250.1947 - accuracy: 0.3869 - val_loss: 100.1371 - val_accuracy: 0.4919
Epoch 2/1000
429/429 [==============================] - 0s 85us/step - loss: 69.9448 - accuracy: 0.6643 - val_loss: 70.9306 - val_accuracy: 0.6973
Epoch 3/1000
429/429 [==============================] - 0s 89us/step - loss: 34.8556 - accuracy: 0.6131 - val_loss: 11.6855 - val_accuracy: 0.6541
Epoch 4/1000
429/429 [==============================] - 0s 98us/step - loss: 12.0992 - accuracy: 0.5315 - val_loss: 8.9535 - val_accuracy: 0.5676
Epoch 5/1000
429/429 [==============================] - 0s 76us/step - loss: 9.1718 - accuracy: 0.5571 - val_loss: 8.4046 - val_accuracy: 0.5297
Epoch 6/1000
429/429 [==============================] - 0s 83us/step - loss: 7.9046 - accuracy: 0.5478 - val_loss: 7.7521 - val_accuracy: 0.4649
Epoch 7/1000
429/429 [==============================] - 0s 106us/step - lo

429/429 [==============================] - 0s 80us/step - loss: 3.6051 - accuracy: 0.6061 - val_loss: 3.6142 - val_accuracy: 0.5730
Epoch 57/1000
429/429 [==============================] - 0s 76us/step - loss: 5.5073 - accuracy: 0.6270 - val_loss: 5.5443 - val_accuracy: 0.6973
Epoch 58/1000
429/429 [==============================] - 0s 87us/step - loss: 5.4985 - accuracy: 0.6014 - val_loss: 10.4482 - val_accuracy: 0.4541
Epoch 59/1000
429/429 [==============================] - 0s 95us/step - loss: 8.2355 - accuracy: 0.5594 - val_loss: 2.9566 - val_accuracy: 0.6973
Epoch 60/1000
429/429 [==============================] - 0s 76us/step - loss: 2.0559 - accuracy: 0.6690 - val_loss: 1.7764 - val_accuracy: 0.6811
Epoch 61/1000
429/429 [==============================] - 0s 84us/step - loss: 3.9787 - accuracy: 0.6084 - val_loss: 4.3182 - val_accuracy: 0.6865
Epoch 62/1000
429/429 [==============================] - 0s 91us/step - loss: 9.0743 - accuracy: 0.5734 - val_loss: 4.5007 - val_accuracy

429/429 [==============================] - 0s 97us/step - loss: 4.8371 - accuracy: 0.6084 - val_loss: 8.6114 - val_accuracy: 0.6865
Epoch 113/1000
429/429 [==============================] - 0s 92us/step - loss: 4.0472 - accuracy: 0.6434 - val_loss: 10.7628 - val_accuracy: 0.3514
Epoch 114/1000
429/429 [==============================] - 0s 75us/step - loss: 6.8294 - accuracy: 0.5664 - val_loss: 9.6692 - val_accuracy: 0.6865
Epoch 115/1000
429/429 [==============================] - 0s 96us/step - loss: 9.0353 - accuracy: 0.6154 - val_loss: 9.0942 - val_accuracy: 0.6919
Epoch 116/1000
429/429 [==============================] - 0s 75us/step - loss: 5.0342 - accuracy: 0.6364 - val_loss: 3.4806 - val_accuracy: 0.5297
Epoch 117/1000
429/429 [==============================] - 0s 80us/step - loss: 4.7780 - accuracy: 0.6224 - val_loss: 4.9916 - val_accuracy: 0.6919
Epoch 118/1000
429/429 [==============================] - 0s 85us/step - loss: 2.9413 - accuracy: 0.6340 - val_loss: 2.6391 - val_ac

Epoch 168/1000
429/429 [==============================] - 0s 74us/step - loss: 2.9422 - accuracy: 0.6340 - val_loss: 3.9981 - val_accuracy: 0.4811
Epoch 169/1000
429/429 [==============================] - 0s 85us/step - loss: 1.8766 - accuracy: 0.6643 - val_loss: 3.2721 - val_accuracy: 0.5351
Epoch 170/1000
429/429 [==============================] - 0s 92us/step - loss: 1.4157 - accuracy: 0.7086 - val_loss: 3.2914 - val_accuracy: 0.5027
Epoch 171/1000
429/429 [==============================] - 0s 96us/step - loss: 2.2225 - accuracy: 0.6340 - val_loss: 2.2014 - val_accuracy: 0.6973
Epoch 172/1000
429/429 [==============================] - 0s 92us/step - loss: 1.9417 - accuracy: 0.6713 - val_loss: 1.9825 - val_accuracy: 0.6000
Epoch 173/1000
429/429 [==============================] - 0s 93us/step - loss: 2.1813 - accuracy: 0.6573 - val_loss: 2.5337 - val_accuracy: 0.6919
Epoch 174/1000
429/429 [==============================] - 0s 111us/step - loss: 3.1061 - accuracy: 0.5921 - val_loss: 

...

Epoch 724/1000
429/429 [==============================] - 0s 100us/step - loss: 2.2796 - accuracy: 0.7203 - val_loss: 1.9395 - val_accuracy: 0.7514
Epoch 725/1000
429/429 [==============================] - 0s 104us/step - loss: 1.2248 - accuracy: 0.7762 - val_loss: 3.4996 - val_accuracy: 0.5459
Epoch 726/1000
429/429 [==============================] - 0s 90us/step - loss: 1.7294 - accuracy: 0.7249 - val_loss: 3.3187 - val_accuracy: 0.5351
Epoch 727/1000
429/429 [==============================] - 0s 92us/step - loss: 2.0751 - accuracy: 0.7529 - val_loss: 7.0745 - val_accuracy: 0.3459
Epoch 728/1000
429/429 [==============================] - 0s 67us/step - loss: 2.1574 - accuracy: 0.6946 - val_loss: 2.0655 - val_accuracy: 0.7514
Epoch 729/1000
429/429 [==============================] - 0s 70us/step - loss: 2.0289 - accuracy: 0.7156 - val_loss: 2.2502 - val_accuracy: 0.7405
Epoch 730/1000
429/429 [==============================] - 0s 71us/step - loss: 1.6117 - accuracy: 0.7622 - val_loss:

Epoch 780/1000
429/429 [==============================] - 0s 82us/step - loss: 0.8855 - accuracy: 0.7692 - val_loss: 2.3502 - val_accuracy: 0.7351
Epoch 781/1000
429/429 [==============================] - 0s 85us/step - loss: 3.8338 - accuracy: 0.6503 - val_loss: 3.9182 - val_accuracy: 0.5297
Epoch 782/1000
429/429 [==============================] - 0s 74us/step - loss: 1.9522 - accuracy: 0.7343 - val_loss: 2.6985 - val_accuracy: 0.6216
Epoch 783/1000
429/429 [==============================] - 0s 81us/step - loss: 1.5043 - accuracy: 0.7506 - val_loss: 1.7817 - val_accuracy: 0.7189
Epoch 784/1000
429/429 [==============================] - 0s 98us/step - loss: 1.0873 - accuracy: 0.7552 - val_loss: 1.6962 - val_accuracy: 0.7081
Epoch 785/1000
429/429 [==============================] - 0s 95us/step - loss: 1.2559 - accuracy: 0.7576 - val_loss: 2.3077 - val_accuracy: 0.7297
Epoch 786/1000
429/429 [==============================] - 0s 88us/step - loss: 2.7143 - accuracy: 0.7296 - val_loss: 4

Epoch 836/1000
429/429 [==============================] - 0s 75us/step - loss: 0.7902 - accuracy: 0.8019 - val_loss: 1.7257 - val_accuracy: 0.7622
Epoch 837/1000
429/429 [==============================] - 0s 84us/step - loss: 2.0195 - accuracy: 0.7366 - val_loss: 3.4113 - val_accuracy: 0.5405
Epoch 838/1000
429/429 [==============================] - 0s 90us/step - loss: 1.4066 - accuracy: 0.7040 - val_loss: 1.7473 - val_accuracy: 0.7676
Epoch 839/1000
429/429 [==============================] - 0s 78us/step - loss: 5.2035 - accuracy: 0.6807 - val_loss: 2.8450 - val_accuracy: 0.7027
Epoch 840/1000
429/429 [==============================] - 0s 71us/step - loss: 1.5880 - accuracy: 0.7203 - val_loss: 2.5698 - val_accuracy: 0.7189
Epoch 841/1000
429/429 [==============================] - 0s 73us/step - loss: 3.4273 - accuracy: 0.6807 - val_loss: 2.1109 - val_accuracy: 0.7459
Epoch 842/1000
429/429 [==============================] - 0s 82us/step - loss: 1.7089 - accuracy: 0.7389 - val_loss: 2

Epoch 892/1000
429/429 [==============================] - 0s 94us/step - loss: 1.5848 - accuracy: 0.7273 - val_loss: 1.8545 - val_accuracy: 0.7081
Epoch 893/1000
429/429 [==============================] - 0s 87us/step - loss: 1.5889 - accuracy: 0.7483 - val_loss: 2.3028 - val_accuracy: 0.7568
Epoch 894/1000
429/429 [==============================] - 0s 75us/step - loss: 1.7561 - accuracy: 0.7319 - val_loss: 1.7319 - val_accuracy: 0.7405
Epoch 895/1000
429/429 [==============================] - 0s 71us/step - loss: 1.6930 - accuracy: 0.7692 - val_loss: 5.2232 - val_accuracy: 0.3892
Epoch 896/1000
429/429 [==============================] - 0s 86us/step - loss: 1.5970 - accuracy: 0.7459 - val_loss: 1.8321 - val_accuracy: 0.7351
Epoch 897/1000
429/429 [==============================] - 0s 80us/step - loss: 1.0636 - accuracy: 0.7483 - val_loss: 3.2734 - val_accuracy: 0.5946
Epoch 898/1000
429/429 [==============================] - 0s 96us/step - loss: 1.1252 - accuracy: 0.7716 - val_loss: 3

Epoch 948/1000
429/429 [==============================] - 0s 90us/step - loss: 1.7891 - accuracy: 0.7110 - val_loss: 1.7064 - val_accuracy: 0.7730
Epoch 949/1000
429/429 [==============================] - 0s 77us/step - loss: 1.6876 - accuracy: 0.7552 - val_loss: 2.7720 - val_accuracy: 0.7243
Epoch 950/1000
429/429 [==============================] - 0s 99us/step - loss: 6.9830 - accuracy: 0.6270 - val_loss: 5.8342 - val_accuracy: 0.6973
Epoch 951/1000
429/429 [==============================] - 0s 77us/step - loss: 4.4913 - accuracy: 0.6876 - val_loss: 4.5403 - val_accuracy: 0.4919
Epoch 952/1000
429/429 [==============================] - 0s 83us/step - loss: 1.7200 - accuracy: 0.7436 - val_loss: 2.4299 - val_accuracy: 0.7514
Epoch 953/1000
429/429 [==============================] - 0s 80us/step - loss: 1.5873 - accuracy: 0.7389 - val_loss: 1.7043 - val_accuracy: 0.7730
Epoch 954/1000
429/429 [==============================] - 0s 90us/step - loss: 1.5521 - accuracy: 0.7156 - val_loss: 4

## Model evaluation

In [478]:
predict = model.predict_classes(x_test)
x_test.shape

(367, 15)

## Save model

In [479]:
model.save("loan_predictor.h5")

## Create Submission file

In [480]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission.Loan_Status = predict
sample_submission.to_csv("sample_submision.csv",index=True)
sample_submission.head()

,Loan_ID,Loan_Status
0,LP001015,1
1,LP001022,1
2,LP001031,1
3,LP001035,1
4,LP001051,0


In [481]:
sample_submission.shape
predict.shape

(367, 1)